# LightGBM

### Parametry:

* **abilities**:
    * **cha**
    * **con**
    * **dex**
    * **int**
    * **str**
    * **wis**
    
* **attributes**
    * **hp**
    * **ac**
    * **perception**
* **saves**
    * **fortitude**
    * **reflex**
    * **will**
* **resources**
    * **focus**

### Przewidujemy: *level*

## Loading data

In [1]:
import pandas as pd
import numpy as np
import sys

import os
import pathlib

sys.path.append("../..")
from training.creating_dataset import load_and_preprocess_data

current_path = os.getcwd()
DATASETS_DIR = pathlib.Path(current_path).parent.parent / "pathfinder_2e_data"
DATASET_FILES = [
    "pathfinder-bestiary.db",
    "pathfinder-bestiary-2.db",
    "pathfinder-bestiary-3.db",
]
DATASET_PATHS = [f"{DATASETS_DIR}/{file}" for file in DATASET_FILES]

df = load_and_preprocess_data(
    DATASET_PATHS,
    characteristics=[
        "cha",
        "con",
        "dex",
        "int",
        "str",
        "wis",
        "ac",
        "hp",
        "perception",
        "fortitude",
        "reflex",
        "will",
        "focus",
    ],
)

c:\users\pauli\appdata\local\programs\python\python39\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [2]:
print(df.head())

   cha  con  dex  int  str  wis  ac   hp  perception  fortitude  reflex  will  \
0    1    5    2    1    7    2  29  215          20         22      19    17   
1    1    5    0   -4    9    2  28  220          16         21      14    16   
2   -2    6    3   -4    7    3  25  175          17         20      17    15   
3    6    5    4    6    7    5  41  315          30         30      29    32   
4    1    1    3   -1   -5    1  16   20           6          6      10     4   

   focus                 book  level  
0      1  Pathfinder Bestiary     10  
1      0  Pathfinder Bestiary     10  
2      0  Pathfinder Bestiary      8  
3      1  Pathfinder Bestiary     17  
4      1  Pathfinder Bestiary      1  


## Train test split

### Chronological split

In [3]:
from training.splitting_dataset import split_dataframe
import os

os.chdir("../../training")
X_train, X_test, y_train, y_test = split_dataframe(df, test_size=0.31)
os.chdir("../notebooks/models")

In [4]:
X_train = X_train.drop(columns=["book"])
X_test = X_test.drop(columns=["book"])

### <span style="color:red">Z jakiegos powodu X_train i y_train ma typy object zamiast int64. Naprawiłabym to w funkcji albo chociaż zrzutowała na inta.</span>

In [5]:
print(X_train.info())
print(X_test.info())
print(y_train.head())
print(y_test.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 841 entries, 0 to 840
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   cha         841 non-null    object
 1   con         841 non-null    object
 2   dex         841 non-null    object
 3   int         841 non-null    object
 4   str         841 non-null    object
 5   wis         841 non-null    object
 6   ac          841 non-null    object
 7   hp          841 non-null    object
 8   perception  841 non-null    object
 9   fortitude   841 non-null    object
 10  reflex      841 non-null    object
 11  will        841 non-null    object
 12  focus       841 non-null    object
dtypes: object(13)
memory usage: 92.0+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 365 entries, 841 to 1205
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   cha         365 non-null    int64
 1   con         365 no

In [6]:
X_train = X_train.astype("int64")
y_train = y_train.astype("int64")

### Random split

In [7]:
from training.splitting_dataset import split_dataframe

(
    X_train_random_split,
    X_test_random_split,
    y_train_random_split,
    y_test_random_split,
) = split_dataframe(df, chronological_split=False)

In [8]:
X_train_random_split = X_train_random_split.drop(columns=["book"])
X_test_random_split = X_test_random_split.drop(columns=["book"])

## LightGBM

In [9]:
# !pip install lightgbm

### With chronological split

In [10]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

params = {
    "boosting_type": "gbdt",
    "objective": "regression",
    "metric": {"l2", "l1"},
}

gbm = lgb.train(
    params,
    lgb_train,
    num_boost_round=20,
    valid_sets=lgb_eval,
    callbacks=[lgb.early_stopping(stopping_rounds=5)],
)


# save model to file
# gbm.save_model('model.txt')


y_pred_train = gbm.predict(X_train, num_iteration=gbm.best_iteration)
y_pred_test = gbm.predict(X_test, num_iteration=gbm.best_iteration)


mse = mean_squared_error(y_test, y_pred_test)
rmse = mean_squared_error(y_test, y_pred_test, squared=False)
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000325 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 379
[LightGBM] [Info] Number of data points in the train set: 841, number of used features: 13
[LightGBM] [Info] Start training from score 6.966706
[1]	valid_0's l1: 4.23939	valid_0's l2: 26.9105
Training until validation scores don't improve for 5 rounds
[2]	valid_0's l1: 3.83208	valid_0's l2: 21.9787
[3]	valid_0's l1: 3.46516	valid_0's l2: 18.02
[4]	valid_0's l1: 3.13596	valid_0's l2: 14.7946
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5]	valid_0's l1: 2.84126	valid_0's l2: 12.1626
[6]	valid_0's l1: 2.57347	valid_0's l2: 10.02
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[7]	valid_0's l1: 2.33115	valid_0's l2: 8.25861
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[8]	valid_0's l1: 2.11654	valid_0's l2: 6

Takie sobie te wyniki bez tuningu.

### With random split

In [11]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

lgb_train = lgb.Dataset(X_train_random_split, y_train_random_split)
lgb_eval = lgb.Dataset(X_test_random_split, y_test_random_split, reference=lgb_train)

params = {
    "boosting_type": "gbdt",
    "objective": "regression",
    "metric": {"l2", "l1"},
}

gbm = lgb.train(
    params,
    lgb_train,
    num_boost_round=20,
    valid_sets=lgb_eval,
    callbacks=[lgb.early_stopping(stopping_rounds=5)],
)


# save model to file
# gbm.save_model('model.txt')


y_pred_train_random_split = gbm.predict(
    X_train_random_split, num_iteration=gbm.best_iteration
)
y_pred_test_random_split = gbm.predict(
    X_test_random_split, num_iteration=gbm.best_iteration
)


mse = mean_squared_error(y_test_random_split, y_pred_test_random_split)
rmse = mean_squared_error(y_test_random_split, y_pred_test_random_split, squared=False)
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000256 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 392
[LightGBM] [Info] Number of data points in the train set: 904, number of used features: 13
[LightGBM] [Info] Start training from score 7.162611
[1]	valid_0's l1: 4.13458	valid_0's l2: 25.0309
Training until validation scores don't improve for 5 rounds
[2]	valid_0's l1: 3.7247	valid_0's l2: 20.3232
[3]	valid_0's l1: 3.35623	valid_0's l2: 16.5022
[4]	valid_0's l1: 3.02251	valid_0's l2: 13.3914
[5]	valid_0's l1: 2.72647	valid_0's l2: 10.8966
[6]	valid_0's l1: 2.45813	valid_0's l2: 8.87112
[7]	valid_0's l1: 2.21691	valid_0's l2: 7.22977
[8]	valid_0's l1: 1.99825	valid_0's l2: 5.89496
[9]	valid_0's l1: 1.80389	valid_0's l2: 4.82388
[10]	valid_0's l1: 1.62726	valid_0's l2: 3.93982
[11]	valid_0's l1: 1.47121	valid_0's l2: 3.23764
[12]	valid_0's l1: 1.32886	valid_0's l2: 2.65446
[13]	valid_0's l1: 1

## Tuning

In [12]:
!pip install lightgbm numpy optuna scikit-learn --user

In [13]:
# "metric":{"l1", "l2"}

In [14]:
import optuna.integration.lightgbm as lgb
from lightgbm import early_stopping
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold

# dtrain = lgb.Dataset(data, label=target)
lgb_train = lgb.Dataset(X_train, y_train)
# lgb_eval = lgb.Dataset(X_test_random_split, y_test_random_split, reference=lgb_train)

params = {
    "boosting_type": "gbdt",
    "objective": "regression",
    "metric": "l2",
    "verbosity": -1,
}

# params = {
#     "objective": "binary",
#     "metric": "binary_logloss",
#     "verbosity": -1,
#     "boosting_type": "gbdt",
# }

# gbm = lgb.train(
#     params,
#     lgb_train,
#     num_boost_round=20,
#     valid_sets=lgb_eval,
#     callbacks=[lgb.early_stopping(stopping_rounds=5)],
# )

tuner = lgb.LightGBMTunerCV(
    params,
    lgb_train,
    folds=KFold(n_splits=5),
    # callbacks=[LightGBMPruningCallback(trial, "l1")],
    callbacks=[early_stopping(100)],
    return_cvbooster=True,
)

tuner.run()

print("Best score:", tuner.best_score)
best_params = tuner.best_params
print("Best params:", best_params)
print("  Params: ")
for key, value in best_params.items():
    print("    {}: {}".format(key, value))

[I 2023-08-06 22:26:49,015] A new study created in memory with name: no-name-7ecee550-983f-463f-9153-b081d205eeca
feature_fraction, val_score: inf:   0%|                                                          | 0/7 [00:00<?, ?it/s]

[1]	cv_agg's l2: 24.8329 + 2.01746
Training until validation scores don't improve for 100 rounds
[2]	cv_agg's l2: 20.2636 + 1.70897
[3]	cv_agg's l2: 16.5483 + 1.38053
[4]	cv_agg's l2: 13.5352 + 1.11067
[5]	cv_agg's l2: 11.0181 + 0.875369
[6]	cv_agg's l2: 8.98728 + 0.711984
[7]	cv_agg's l2: 7.34859 + 0.573234
[8]	cv_agg's l2: 6.00476 + 0.472122
[9]	cv_agg's l2: 4.92328 + 0.389139
[10]	cv_agg's l2: 4.04255 + 0.310993
[11]	cv_agg's l2: 3.33183 + 0.25797
[12]	cv_agg's l2: 2.75405 + 0.214363
[13]	cv_agg's l2: 2.27686 + 0.180576
[14]	cv_agg's l2: 1.89105 + 0.152
[15]	cv_agg's l2: 1.57783 + 0.135637
[16]	cv_agg's l2: 1.3275 + 0.11161
[17]	cv_agg's l2: 1.11985 + 0.0986648
[18]	cv_agg's l2: 0.949696 + 0.0885509
[19]	cv_agg's l2: 0.811407 + 0.0800956
[20]	cv_agg's l2: 0.700042 + 0.0714654
[21]	cv_agg's l2: 0.611263 + 0.0651101
[22]	cv_agg's l2: 0.537298 + 0.0611577
[23]	cv_agg's l2: 0.476511 + 0.0555232
[24]	cv_agg's l2: 0.424759 + 0.0523963
[25]	cv_agg's l2: 0.383332 + 0.0486521
[26]	cv_agg's l

[220]	cv_agg's l2: 0.165298 + 0.0231802
[221]	cv_agg's l2: 0.165345 + 0.0233203
[222]	cv_agg's l2: 0.165435 + 0.0232882
[223]	cv_agg's l2: 0.165474 + 0.0232394
[224]	cv_agg's l2: 0.16545 + 0.0231882
[225]	cv_agg's l2: 0.165489 + 0.0231905
[226]	cv_agg's l2: 0.165604 + 0.0231868
[227]	cv_agg's l2: 0.165585 + 0.0230294
[228]	cv_agg's l2: 0.165477 + 0.022934
[229]	cv_agg's l2: 0.165552 + 0.0229437
[230]	cv_agg's l2: 0.165556 + 0.0229266
[231]	cv_agg's l2: 0.165628 + 0.0229494
[232]	cv_agg's l2: 0.16573 + 0.0229458
[233]	cv_agg's l2: 0.16573 + 0.022947
[234]	cv_agg's l2: 0.165748 + 0.0229874
[235]	cv_agg's l2: 0.165749 + 0.0229629
[236]	cv_agg's l2: 0.165771 + 0.0229958
[237]	cv_agg's l2: 0.16573 + 0.0230168
[238]	cv_agg's l2: 0.165714 + 0.0230904
[239]	cv_agg's l2: 0.165766 + 0.0231209
[240]	cv_agg's l2: 0.165721 + 0.0231228
[241]	cv_agg's l2: 0.165715 + 0.0231201
[242]	cv_agg's l2: 0.165783 + 0.0231104
[243]	cv_agg's l2: 0.165902 + 0.0231145
[244]	cv_agg's l2: 0.165944 + 0.0232479
[245]	

feature_fraction, val_score: 0.165054:  14%|######4                                      | 1/7 [00:01<00:06,  1.11s/it]

[273]	cv_agg's l2: 0.165641 + 0.0231674
[274]	cv_agg's l2: 0.165666 + 0.0231768
[275]	cv_agg's l2: 0.16567 + 0.0232082
[276]	cv_agg's l2: 0.165794 + 0.0232815
[277]	cv_agg's l2: 0.165942 + 0.0232878
[278]	cv_agg's l2: 0.165923 + 0.023314
[279]	cv_agg's l2: 0.165798 + 0.0232367
[280]	cv_agg's l2: 0.165738 + 0.0231422
[281]	cv_agg's l2: 0.165744 + 0.023277
[282]	cv_agg's l2: 0.165761 + 0.0231466
[283]	cv_agg's l2: 0.165779 + 0.023185
[284]	cv_agg's l2: 0.165807 + 0.0230412
[285]	cv_agg's l2: 0.165765 + 0.0230638
[286]	cv_agg's l2: 0.165781 + 0.023084
[287]	cv_agg's l2: 0.165725 + 0.0231596
[288]	cv_agg's l2: 0.165797 + 0.0231936
[289]	cv_agg's l2: 0.16581 + 0.0231533
[290]	cv_agg's l2: 0.165777 + 0.023192
[291]	cv_agg's l2: 0.165898 + 0.0232338
[292]	cv_agg's l2: 0.165849 + 0.023256
[293]	cv_agg's l2: 0.165927 + 0.0232674
[294]	cv_agg's l2: 0.165873 + 0.0233558
[295]	cv_agg's l2: 0.165815 + 0.0233788
[296]	cv_agg's l2: 0.1658 + 0.0234295
[297]	cv_agg's l2: 0.165783 + 0.0234183
[298]	cv_a

...

min_data_in_leaf, val_score: 0.157198:  80%|####################################         | 4/5 [00:04<00:00,  1.03it/s]

[317]	cv_agg's l2: 0.176856 + 0.0320119
[318]	cv_agg's l2: 0.177005 + 0.0320026
[319]	cv_agg's l2: 0.177019 + 0.0319008
[320]	cv_agg's l2: 0.176994 + 0.0319385
[321]	cv_agg's l2: 0.177017 + 0.0319184
[322]	cv_agg's l2: 0.177104 + 0.0320366
[323]	cv_agg's l2: 0.177152 + 0.0319926
[324]	cv_agg's l2: 0.177058 + 0.0320505
[325]	cv_agg's l2: 0.177023 + 0.0320457
[326]	cv_agg's l2: 0.176985 + 0.0320487
[327]	cv_agg's l2: 0.17696 + 0.0320034
[328]	cv_agg's l2: 0.176851 + 0.0319641
Early stopping, best iteration is:
[228]	cv_agg's l2: 0.157198 + 0.0251478
[1]	cv_agg's l2: 24.7982 + 1.95504
[2]	cv_agg's l2: 20.1951 + 1.63424
[3]	cv_agg's l2: 16.4539 + 1.31388
[4]	cv_agg's l2: 13.3978 + 1.06802
[5]	cv_agg's l2: 10.9371 + 0.858161
[6]	cv_agg's l2: 8.9414 + 0.742235
[7]	cv_agg's l2: 7.32899 + 0.607908
[8]	cv_agg's l2: 6.01129 + 0.490829
[9]	cv_agg's l2: 4.9236 + 0.418085
[10]	cv_agg's l2: 4.03548 + 0.344897
[11]	cv_agg's l2: 3.32048 + 0.301174
[12]	cv_agg's l2: 2.74377 + 0.264293
[13]	cv_agg's l2:

[202]	cv_agg's l2: 0.147294 + 0.0308459
[203]	cv_agg's l2: 0.147305 + 0.0308163
[204]	cv_agg's l2: 0.147274 + 0.0308142
[205]	cv_agg's l2: 0.147238 + 0.0307736
[206]	cv_agg's l2: 0.14723 + 0.0308527
[207]	cv_agg's l2: 0.147132 + 0.0308262
[208]	cv_agg's l2: 0.147177 + 0.0308763
[209]	cv_agg's l2: 0.147171 + 0.0308163
[210]	cv_agg's l2: 0.147185 + 0.0308633
[211]	cv_agg's l2: 0.147165 + 0.0308556
[212]	cv_agg's l2: 0.147129 + 0.0308597
[213]	cv_agg's l2: 0.147087 + 0.0309133
[214]	cv_agg's l2: 0.147058 + 0.0309659
[215]	cv_agg's l2: 0.147055 + 0.0309674
[216]	cv_agg's l2: 0.147028 + 0.031031
[217]	cv_agg's l2: 0.147039 + 0.0310005
[218]	cv_agg's l2: 0.147049 + 0.0310637
[219]	cv_agg's l2: 0.147037 + 0.0311231
[220]	cv_agg's l2: 0.147032 + 0.0311822
[221]	cv_agg's l2: 0.14708 + 0.0311915
[222]	cv_agg's l2: 0.146995 + 0.0311713
[223]	cv_agg's l2: 0.146989 + 0.0311054
[224]	cv_agg's l2: 0.147076 + 0.031138
[225]	cv_agg's l2: 0.146996 + 0.0310863
[226]	cv_agg's l2: 0.146989 + 0.0311614
[227

min_data_in_leaf, val_score: 0.146879: 100%|#############################################| 5/5 [00:05<00:00,  1.13s/it]

[317]	cv_agg's l2: 0.147649 + 0.0313748
[318]	cv_agg's l2: 0.147647 + 0.0313729
[319]	cv_agg's l2: 0.147605 + 0.0313562
[320]	cv_agg's l2: 0.147592 + 0.0313583
[321]	cv_agg's l2: 0.1476 + 0.0313301
[322]	cv_agg's l2: 0.147632 + 0.031331
[323]	cv_agg's l2: 0.14767 + 0.0313554
[324]	cv_agg's l2: 0.147713 + 0.0313767
[325]	cv_agg's l2: 0.147739 + 0.0314344
[326]	cv_agg's l2: 0.147748 + 0.0314592
[327]	cv_agg's l2: 0.147764 + 0.0314406
Early stopping, best iteration is:
[227]	cv_agg's l2: 0.146879 + 0.0311168
Best score: 0.14687854586793855
Best params: {'boosting_type': 'gbdt', 'objective': 'regression', 'metric': 'l2', 'verbosity': -1, 'feature_pre_filter': False, 'lambda_l1': 0.004287905350384767, 'lambda_l2': 0.00037389395328371155, 'num_leaves': 31, 'feature_fraction': 0.4, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 10}
  Params: 
    boosting_type: gbdt
    objective: regression
    metric: l2
    verbosity: -1
    feature_pre_filter: False
    lambda_l1: 0.0042

In [15]:
"""
Optuna example that optimizes a classifier configuration for cancer dataset using LightGBM tuner.

In this example, we optimize the cross-validated log loss of cancer detection.

"""
import optuna.integration.lightgbm as lgb

from lightgbm import early_stopping
from lightgbm import log_evaluation
import sklearn.datasets
from sklearn.model_selection import KFold


if __name__ == "__main__":
    data, target = sklearn.datasets.load_breast_cancer(return_X_y=True)
    dtrain = lgb.Dataset(data, label=target)

    params = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
    }

    tuner = lgb.LightGBMTunerCV(
        params,
        dtrain,
        folds=KFold(n_splits=3),
        callbacks=[early_stopping(100), log_evaluation(100)],
    )

    tuner.run()

    print("Best score:", tuner.best_score)
    best_params = tuner.best_params
    print("Best params:", best_params)
    print("  Params: ")
    for key, value in best_params.items():
        print("    {}: {}".format(key, value))

ImportError: cannot import name 'log_evaluation' from 'lightgbm' (C:\Users\pauli\AppData\Roaming\Python\Python39\site-packages\lightgbm\__init__.py)

In [ ]:
import sklearn.datasets
from sklearn.model_selection import KFold

import optuna.integration.lightgbm as lgb


if __name__ == "__main__":
    data, target = sklearn.datasets.fetch_kddcup99(return_X_y=True)
    dtrain = lgb.Dataset(data, label=target)

    params = {"objective": "binary", "metric": "binary_logloss", "verbosity": -1}

    tuner = lgb.LightGBMTunerCV(
        params,
        dtrain,
        time_budget=60,
        verbose_eval=0,
        early_stopping_rounds=50,
        folds=KFold(n_splits=3),
    )

    tuner.run()

    print("Best score:", tuner.best_score)
    best_params = tuner.best_params
    print("Best params:", best_params)
    print("  Params: ")
    for key, value in best_params.items():
        print("    {}: {}".format(key, value))